<a href="https://colab.research.google.com/github/amolaka/DS-3001---Voting-Project/blob/main/Voting_Project_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voting Project Paper
## Predicting 2024 Virginia Presidential Election Outcomes

## Summary

This project aims to predict the winner of the 2024 presidential election in the state of Virginia. The approach to developing this predicitive involves using voting data from presidential elections for Virginia from 2000 to 2020 and NHGIS county data, which contains county stats from every state in the country.
This data will then be cleaned and preprocessed, along with undergoing some EDA, to help determine which variables will be the most useful to creating a model. We chose to focus on the variables pertaining to income, age, education, and race statistics to build a model. The models we chose to explore were kNN, linear regression, and random forests. From these models, the best proved to be ____ with __ __(supporting stats that provide Quantitative Information About Our Prediction's Precision)__  ,____. Additionally, county adjacencies data was used to create chloropleth plots that help visualize these results over the a map of Virginia.

___ any other results

## Data

The Voting Data file with information on presidential elections for Virginia from 2000 to 2020 includes variables such as:


*   year
*   county
* candidate
* party
* candidate votes
* total votes

The NHGIS county data demographic and economic variables we chose to focus on were


*   income (grouped into 0-50K, 50k-100k, and 100k+)
*   education (grouped by male and female as bachelor's degree and up or no degree)
* age (grouped into 21-50 and 50+)
* race (hispanic or latino, white, black, asian)







## Cleaning the Data & Challenges

When cleaning the Voting and county adjacencies data, cleaning consisted of dropping NA's. Cleaning the NHGIS county data came with a few more challenges.To start, the second row of the dataframe had to be made as the header since that was the row that contained the actual descriptive column names. If left as was, this would have interfered with data types of the variables of interest, such as them not being all numerical. After removing that first row, there were still an issue with the data types, so certain variables had to be changed from objects to integer data types. Additionally, some of the county data files, like ./0002_ds176_20105_county_E.csv contained weird looking extra rows at the end had to be dropped. However, after looking further into these files, some did not contain data from the state of Virginia, which were what we desired, so not all of these county files ended up even being used.

## Data Preprocessing

After isolating the county data from the state of interest, Virginia, the county dataframes files from different years were combined into a single dataframe. From here, then data was then cleaned further by dropping NA's.